In [1]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever

c:\Users\91830\anaconda3\envs\genaienv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
    
]

In [3]:
all_docs

[Document(metadata={'source': 'H1'}, page_content='Regular walking boosts heart health and can reduce symptoms of depression.'),
 Document(metadata={'source': 'H2'}, page_content='Consuming leafy greens and fruits helps detox the body and improve longevity.'),
 Document(metadata={'source': 'H3'}, page_content='Deep sleep is crucial for cellular repair and emotional regulation.'),
 Document(metadata={'source': 'H4'}, page_content='Mindfulness and controlled breathing lower cortisol and improve mental clarity.'),
 Document(metadata={'source': 'H5'}, page_content='Drinking sufficient water throughout the day helps maintain metabolism and energy.'),
 Document(metadata={'source': 'I1'}, page_content='The solar energy system in modern homes helps balance electricity demand.'),
 Document(metadata={'source': 'I2'}, page_content='Python balances readability with power, making it a popular system design language.'),
 Document(metadata={'source': 'I3'}, page_content='Photosynthesis enables plants

In [4]:
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
vectorstore=FAISS.from_documents(
    documents=all_docs, 
    embedding=embedding_model
)

In [6]:
similarity_retriever=vectorstore.as_retriever(
    search_type='similarity', 
    search_kwargs={'k':5}
)

In [7]:
multi_query_retriever=MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatGoogleGenerativeAI(model='gemini-2.5-pro')
)

# the search_kwargs (like {"k": 5}) are not for the MultiQueryRetriever itself, but for the base retriever (vectorstore.as_retriever()).
# So here: "k": 5 → means “for each query(created by llm), retrieve top 5 most similar documents from the vectorstore.”
# means the final result of the multiqueryretriever can exceed 5

In [8]:
# Query
query = "How to improve energy levels and maintain balance?"

In [9]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multi_query_retriever.invoke(query)

In [ ]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)
    
print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 5 ---
Photosynthesis enables plants to produce energy by converting sunlight.
*

--- Result 1 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 2 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 3 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 4 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 5 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 6 ---
The solar energy system in mo